In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from itertools import combinations
import os

# Set device (multi-GPU setup)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_gpus = torch.cuda.device_count()
print(f"Using {num_gpus} GPU(s)")

# Load and inspect the data
data = np.load("/Data/MI_PHYSICS/Double-Pendulum-Simulation/test1/pendulum_data360.npz")
keys = list(data.keys())
print("Keys in the .npz file:", keys)


Using 2 GPU(s)
Keys in the .npz file: ['Time', 'Theta1', 'Theta2', 'X1', 'Y1', 'X2', 'Y2', 'Energy']


In [2]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Linear(32, 2)  # Bottleneck
        )
        self.decoder = nn.Sequential(
            nn.Linear(2, 32),
            nn.ReLU(),
            nn.Linear(32, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


In [20]:
from tqdm.notebook import tqdm
from itertools import combinations

# Create a tqdm progress bar for overall progress
total_combos = sum(1 for r in range(2, len(keys)+1) for _ in combinations(keys, r))
with tqdm(total=total_combos, desc="Overall Progress", leave=True, dynamic_ncols=True) as overall_bar:
    
    # Iterate over combinations sizes
    for r in range(2, len(keys)+1):
        combos = list(combinations(keys, r))
        
        # Progress bar for current combo size
        with tqdm(total=len(combos), desc=f"Combo size {r}", leave=False, dynamic_ncols=True) as combo_bar:
            
            for combo in combos:
                try:
                    selected_data = np.hstack([data[k].reshape(data[k].shape[0], -1) for k in combo])
                except Exception as e:
                    print(f"Skipping {combo} due to shape mismatch: {e}")
                    continue

                selected_data = (selected_data - selected_data.mean(axis=0)) / (selected_data.std(axis=0) + 1e-8)

                tensor_data = torch.tensor(selected_data, dtype=torch.float32)
                dataset = TensorDataset(tensor_data)
                loader = DataLoader(dataset, batch_size=256, shuffle=True)

                input_dim = selected_data.shape[1]
                model = Autoencoder(input_dim)

                if num_gpus > 1:
                    model = nn.DataParallel(model)
                model.to(device)

                criterion = nn.MSELoss()
                optimizer = optim.Adam(model.parameters(), lr=1e-3)

                combo_name = "_".join(combo)
                print(f"\nTraining on combo: {combo_name}")

                # Progress bar for epochs
                with tqdm(range(20), desc="Epochs", leave=False, smoothing=True, dynamic_ncols=False) as epoch_bar:
                    for epoch in epoch_bar:
                        total_loss = 0
                        for batch in loader:
                            x = batch[0].to(device)
                            optimizer.zero_grad()
                            output = model(x)
                            loss = criterion(output, x)
                            loss.backward()
                            optimizer.step()
                            total_loss += loss.item()

                        epoch_bar.set_postfix({"Loss": total_loss/len(loader)}, refresh=True)

                        torch.save(model, f"autoencoder_{combo_name}.pt")

                        model.eval()  # Switch model to evaluation mode
                        with torch.no_grad():  # Disable gradient calculation for evaluation
                            total_loss = 0
                            for batch in loader:
                                x = batch[0].to(device)
                                output = model(x)
                                loss = criterion(output, x)
                                total_loss += loss.item()

                        # Print evaluation results
                        print(f"Evaluation Loss for {combo_name}: {total_loss/len(loader):.6f}")
                        print("---------------------")

                # Update combo progress
                combo_bar.update(1)
            
        # Update the overall progress
        overall_bar.update(1)


Overall Progress:   0%|          | 0/247 [00:00<?, ?it/s]

Combo size 2:   0%|          | 0/28 [00:00<?, ?it/s]


Training on combo: Time_Theta1


Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

Evaluation Loss for Time_Theta1: 0.000019
---------------------
Evaluation Loss for Time_Theta1: 0.000031
---------------------
Evaluation Loss for Time_Theta1: 0.000019
---------------------
Evaluation Loss for Time_Theta1: 0.000023
---------------------
Evaluation Loss for Time_Theta1: 0.000006
---------------------
Evaluation Loss for Time_Theta1: 0.000069
---------------------
Evaluation Loss for Time_Theta1: 0.000005
---------------------
Evaluation Loss for Time_Theta1: 0.000023
---------------------
Evaluation Loss for Time_Theta1: 0.000019
---------------------
Evaluation Loss for Time_Theta1: 0.000029
---------------------
Evaluation Loss for Time_Theta1: 0.000008
---------------------
Evaluation Loss for Time_Theta1: 0.000004
---------------------
Evaluation Loss for Time_Theta1: 0.000003
---------------------
Evaluation Loss for Time_Theta1: 0.000011
---------------------
Evaluation Loss for Time_Theta1: 0.000036
---------------------
Evaluation Loss for Time_Theta1: 0.00000

Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

Evaluation Loss for Time_Theta2: 0.000004
---------------------
Evaluation Loss for Time_Theta2: 0.000010
---------------------
Evaluation Loss for Time_Theta2: 0.000015
---------------------
Evaluation Loss for Time_Theta2: 0.000001
---------------------
Evaluation Loss for Time_Theta2: 0.000010
---------------------
Evaluation Loss for Time_Theta2: 0.000001
---------------------
Evaluation Loss for Time_Theta2: 0.000029
---------------------
Evaluation Loss for Time_Theta2: 0.000001
---------------------
Evaluation Loss for Time_Theta2: 0.000118
---------------------
Evaluation Loss for Time_Theta2: 0.000044
---------------------
Evaluation Loss for Time_Theta2: 0.000038
---------------------
Evaluation Loss for Time_Theta2: 0.000018
---------------------
Evaluation Loss for Time_Theta2: 0.000008
---------------------
Evaluation Loss for Time_Theta2: 0.000004
---------------------
Evaluation Loss for Time_Theta2: 0.000000
---------------------
Evaluation Loss for Time_Theta2: 0.00004

Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

Evaluation Loss for Time_X1: 0.000018
---------------------


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px
import pandas as pd

# Plot all latent space visualizations
for r in range(2, len(keys)+1):
    for combo in combinations(keys, r):
        try:
            selected_data = np.hstack([data[k].reshape(data[k].shape[0], -1) for k in combo])
        except Exception as e:
            print(f"Skipping plot for {combo} due to shape mismatch: {e}")
            continue

        # Normalize
        selected_data = (selected_data - selected_data.mean(axis=0)) / (selected_data.std(axis=0) + 1e-8)
        tensor_data = torch.tensor(selected_data, dtype=torch.float32)

        # Build same model and load saved weights
        input_dim = selected_data.shape[1]
        model = Autoencoder3D(input_dim)
        model_path = f"autoencoder_{'_'.join(combo)}.pt"

        if num_gpus > 1:
            model = nn.DataParallel(model)
        model.to(device)

        try:
            model.load_state_dict(torch.load(model_path))
        except Exception as e:
            print(f"Failed to load model for {combo}: {e}")
            continue

        model.eval()
        with torch.no_grad():
            encoded = model.module.encode(tensor_data.to(device)).cpu().numpy() if isinstance(model, nn.DataParallel) else model.encode(tensor_data.to(device)).cpu().numpy()

        # ---- 3D Plot using Matplotlib ----
        fig = plt.figure(figsize=(6, 5))
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(encoded[:, 0], encoded[:, 1], encoded[:, 2], s=8, alpha=0.6)
        ax.set_title(f"3D Encoded Space - {combo}")
        ax.set_xlabel("Z1")
        ax.set_ylabel("Z2")
        ax.set_zlabel("Z3")
        plt.tight_layout()
        plt.show()

        # ---- Interactive 3D Plot using Plotly ----
        df = pd.DataFrame(encoded, columns=["Z1", "Z2", "Z3"])
        df["Index"] = df.index

        fig = px.scatter_3d(
            df, x="Z1", y="Z2", z="Z3",
            color="Index",
            title=f"Interactive 3D - {' + '.join(combo)}",
            opacity=0.8,
            width=800,
            height=700
        )
        fig.update_traces(marker=dict(size=4))
        fig.update_layout(scene=dict(
            xaxis_title='Z1',
            yaxis_title='Z2',
            zaxis_title='Z3'
        ))
        fig.show()
